# Imports and Functions

In [1]:
import os
import neo4j
import numpy as np
import pandas as pd
# from IPython.display import display
import json

In [2]:
driver = neo4j.GraphDatabase.driver(uri="neo4j://neo4j:7687", auth=("neo4j","ucb_mids_w205"))
session = driver.session(database="neo4j")

In [3]:
# consolidate functions
def my_neo4j_run_query_pandas(query, **kwargs):
    "run a query and return the results in a pandas dataframe"
    result = session.run(query, **kwargs)
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())
    
    return df
    
def neo4j_to_pandas(query, query_type=None):
    """Takes a neo4j query and returns a dataframe of the result.
    Args:
        query_type = 'nodes' changes the datatype of relevant columns to float
    """
    print("  Query Results:")
    print("-------------------------")

    df = my_neo4j_run_query_pandas(query)
#     if query_type == 'nodes':
#         df[['capacity_mw', 'capacity_mtpa', 'capacity_nominal_crude_steel_capacity_ttpa']] = (
#             df[['capacity_mw', 'capacity_mtpa', 'capacity_nominal_crude_steel_capacity_ttpa']].astype(float))
    
    print(df.shape)
    display(df.head())
    
    return df

# Bottom Up

- United Arab Emirates

In [ ]:
# via energy project nodes of a certain country upwards; query can probably be simplified!
# grab nodes
query = """
MATCH (n:Energy_Project { country: "United Arab Emirates" })
RETURN DISTINCT n.name AS name,
                n.name AS id,
                n.country AS country,
                n.energy_project_type AS type,
                COALESCE(n.status, 'Unknown') AS status,
                COALESCE(n.publicly_listed, 'Unknown') AS publicly_listed,
                COALESCE(n.capcity_mw, 0) AS capacity_mw,
                COALESCE(n.capacity_mtpa, 0) AS capacity_mtpa,
                COALESCE(n.capacity_nominal_crude_steel_capacity_ttpa, 0) AS capacity_nominal_crude_steel_capacity_ttpa

UNION

MATCH (n:Energy_Project { country: "United Arab Emirates" })-[:SHARE*]->(e:Entity)
RETURN DISTINCT e.name AS name,
                e.id AS id,
                e.country AS country,
                'Entity' AS type,
                COALESCE(e.status, 'Entity') AS status,
                COALESCE(e.publicly_listed, 'Unknown') AS publicly_listed,
                COALESCE(e.capcity_mw, 0) AS capacity_mw,
                COALESCE(e.capacity_mtpa, 0) AS capacity_mtpa,
                COALESCE(e.capacity_nominal_crude_steel_capacity_ttpa, 0) AS capacity_nominal_crude_steel_capacity_ttpa

UNION

MATCH (n:Energy_Project { country: "United Arab Emirates" })-[:SHARE*]->(e:Entity)
MATCH (e)-[:SHARE*]->(m:Entity)
RETURN DISTINCT m.name AS name,
                m.id AS id,
                m.country AS country,
                'Entity' AS type,
                COALESCE(m.status, 'N/A') AS status,
                COALESCE(m.publicly_listed, 'Unknown') AS publicly_listed,
                COALESCE(m.capcity_mw, 0) AS capacity_mw,
                COALESCE(m.capacity_mtpa, 0) AS capacity_mtpa,
                COALESCE(m.capacity_nominal_crude_steel_capacity_ttpa, 0) AS capacity_nominal_crude_steel_capacity_ttpa;


"""
df = neo4j_to_pandas(query)

In [ ]:
df.status = df.status.apply(lambda x: x.lower() if pd.notnull(x) else x)
df.index.name = 'index'
df.to_json('/user/projects/project-3-energy-ownership/code/Data/data_viz/test/UAE_nodes_ep_search.json', orient='records')

In [ ]:
# via energy project nodes of a certain country upwards
# grab relationships
query = """
MATCH (n:Energy_Project {country: "United Arab Emirates"})-[r:SHARE*]->(e:Entity)
UNWIND r AS rel
RETURN DISTINCT startNode(rel).name AS source,
    endNode(rel).id AS target,
    CASE WHEN startNode(rel):Energy_Project THEN 1
        ELSE 0 END AS is_source_energy_project,
    COALESCE(startNode(rel).status, 'Unknown') AS status,
    rel.share AS share

UNION

MATCH (n:Energy_Project {country: "United Arab Emirates"})-[:SHARE*]->(e:Entity)
MATCH (e)-[r:SHARE*]->(m:Entity) 
UNWIND r AS rel
RETURN DISTINCT startNode(rel).id AS source,
    endNode(rel).id AS target,
    CASE WHEN startNode(rel):Energy_Project THEN 1
        ELSE 0 END AS is_source_energy_project,
    COALESCE(startNode(rel).status, 'Entity') AS status,
    rel.share AS share
"""
df = neo4j_to_pandas(query)

In [ ]:
df.status = df.status.apply(lambda x: x.lower() if pd.notnull(x) else x)
df.index.name = 'index'
df.to_json('/user/projects/project-3-energy-ownership/code/Data/data_viz/test/UAE_relationships_ep_search.json', orient='records')

- Nigeria

In [5]:
# via energy project nodes of a certain country upwards; query can probably be simplified!
# grab nodes
query = """
MATCH (n:Energy_Project { country: "Nigeria" })
RETURN DISTINCT n.name AS name,
                n.name AS id,
                CASE WHEN n:Energy_Project THEN 1
                    ELSE 0 END AS is_energy_project,
                n.country AS country,
                n.entity_type AS entity_type,
                CASE WHEN n:Energy_Project THEN n.energy_project_type
                    WHEN n:Entity THEN 'Entity'
                    ELSE 'Unknown' END AS energy_project_type,
                n.legal_entity_type AS legal_entity_type,
                n.energy_project_type AS type,
                COALESCE(n.status, 'Unknown') AS status,
                COALESCE(n.publicly_listed, 'Unknown') AS publicly_listed,
                COALESCE(n.bio_plant_capacity, 0) AS bio_plant_capacity,
                COALESCE(n.coal_mine_capacity, 0) AS coal_mine_capacity,
                COALESCE(n.coal_plant_capacity, 0) AS coal_plant_capacity,
                COALESCE(n.gas_plant_capacity, 0) AS gas_plant_capacity,
                COALESCE(n.steel_plant_capacity, 0) AS steel_plant_capacity,
                COALESCE(n.capacity_mtpa, 0) AS capacity_mtpa,
                COALESCE(n.capacity_mw, 0) AS capacity_mw,
                COALESCE(n.capacity_nominal_crude_steel_capacity_ttpa, 0) AS capacity_nominal_crude_steel_capacity_ttpa

UNION

MATCH (n:Energy_Project { country: "Nigeria" })-[:SHARE*]->(e:Entity)
RETURN DISTINCT e.name AS name,
                e.name AS id,
                CASE WHEN e:Energy_Project THEN 1
                    ELSE 0 END AS is_energy_project,
                e.country AS country,
                e.entity_type AS entity_type,
                CASE WHEN e:Energy_Project THEN e.energy_project_type
                    WHEN e:Entity THEN 'Entity'
                    ELSE 'Unknown' END AS energy_project_type,
                e.legal_entity_type AS legal_entity_type,
                e.energy_project_type AS type,
                COALESCE(e.status, 'Unknown') AS status,
                COALESCE(e.publicly_listed, 'Unknown') AS publicly_listed,
                COALESCE(e.bio_plant_capacity, 0) AS bio_plant_capacity,
                COALESCE(e.coal_mine_capacity, 0) AS coal_mine_capacity,
                COALESCE(e.coal_plant_capacity, 0) AS coal_plant_capacity,
                COALESCE(e.gas_plant_capacity, 0) AS gas_plant_capacity,
                COALESCE(e.steel_plant_capacity, 0) AS steel_plant_capacity,
                COALESCE(e.capacity_mtpa, 0) AS capacity_mtpa,
                COALESCE(e.capacity_mw, 0) AS capacity_mw,
                COALESCE(e.capacity_nominal_crude_steel_capacity_ttpa, 0) AS capacity_nominal_crude_steel_capacity_ttpa

UNION

MATCH (n:Energy_Project { country: "Nigeria" })-[:SHARE*]->(e:Entity)
MATCH (e)-[:SHARE*]->(m:Entity)
RETURN DISTINCT m.name AS name,
                m.name AS id,
                CASE WHEN m:Energy_Project THEN 1
                    ELSE 0 END AS is_energy_project,
                m.country AS country,
                m.entity_type AS entity_type,
                CASE WHEN m:Energy_Project THEN e.energy_project_type
                    WHEN m:Entity THEN 'Entity'
                    ELSE 'Unknown' END AS energy_project_type,
                m.legal_entity_type AS legal_entity_type,
                m.energy_project_type AS type,
                COALESCE(m.status, 'Unknown') AS status,
                COALESCE(m.publicly_listed, 'Unknown') AS publicly_listed,
                COALESCE(m.bio_plant_capacity, 0) AS bio_plant_capacity,
                COALESCE(m.coal_mine_capacity, 0) AS coal_mine_capacity,
                COALESCE(m.coal_plant_capacity, 0) AS coal_plant_capacity,
                COALESCE(m.gas_plant_capacity, 0) AS gas_plant_capacity,
                COALESCE(m.steel_plant_capacity, 0) AS steel_plant_capacity,
                COALESCE(m.capacity_mtpa, 0) AS capacity_mtpa,
                COALESCE(m.capacity_mw, 0) AS capacity_mw,
                COALESCE(m.capacity_nominal_crude_steel_capacity_ttpa, 0) AS capacity_nominal_crude_steel_capacity_ttpa


"""
df = neo4j_to_pandas(query)



  Query Results:
-------------------------
(285, 18)


,name,id,is_energy_project,country,entity_type,energy_project_type,legal_entity_type,type,status,publicly_listed,bio_plant_capacity,coal_mine_capacity,coal_plant_capacity,gas_plant_capacity,steel_plant_capacity,capacity_mtpa,capacity_mw,capacity_nominal_crude_steel_capacity_ttpa
0,Qua Iboe power plant 1,Qua Iboe power plant 1,1,Nigeria,None,Gas or Oil Plant,None,Gas or Oil Plant,announced,Unknown,0,0,0,0,0,0.0,540.0,0
1,Sapele II power station 2-1,Sapele II power station 2-1,1,Nigeria,None,Gas or Oil Plant,None,Gas or Oil Plant,operating,Unknown,0,0,0,0,0,0.0,113.0,0
2,Sapele II power station 2-2,Sapele II power station 2-2,1,Nigeria,None,Gas or Oil Plant,None,Gas or Oil Plant,operating,Unknown,0,0,0,0,0,0.0,113.0,0
3,Sapele II power station 2-3,Sapele II power station 2-3,1,Nigeria,None,Gas or Oil Plant,None,Gas or Oil Plant,operating,Unknown,0,0,0,0,0,0.0,113.0,0
4,Sapele II power station 2-4,Sapele II power station 2-4,1,Nigeria,None,Gas or Oil Plant,None,Gas or Oil Plant,operating,Unknown,0,0,0,0,0,0.0,113.0,0


In [6]:
df.status = df.status.apply(lambda x: x.lower() if pd.notnull(x) else x)
df.index.name = 'index'
df.to_json('/user/projects/project-3-energy-ownership/code/Data/data_viz/test/Nigeria_nodes_bu.json', orient='records')

In [7]:
# via energy project nodes of a certain country upwards
# grab relationships
query = """
MATCH (n:Energy_Project {country: "Nigeria"})-[r:SHARE*]->(e:Entity)
UNWIND r AS rel
RETURN DISTINCT id(rel) AS rel_id,
                startNode(rel).id AS source,
                endNode(rel).id AS target,
                CASE WHEN startNode(rel):Energy_Project THEN 1
                    ELSE 0 END AS is_source_energy_project,
                CASE WHEN startNode(rel):Entity THEN 'Entity'
                    WHEN startNode(rel):Energy_Project THEN startNode(rel).status
                    ELSE 'Unknown' END AS status,
                rel.share AS share

UNION

MATCH (n:Energy_Project {country: "Nigeria"})-[:SHARE*]->(e:Entity)
MATCH (e)-[r:SHARE*]->(m:Entity) 
UNWIND r AS rel
RETURN DISTINCT id(rel) AS rel_id,
                startNode(rel).id AS source,
                endNode(rel).id AS target,
                CASE WHEN startNode(rel):Energy_Project THEN 1
                    ELSE 0 END AS is_source_energy_project,
                CASE WHEN startNode(rel):Entity THEN 'Entity'
                    WHEN startNode(rel):Energy_Project THEN startNode(rel).status
                    ELSE 'Unknown' END AS status,
                rel.share AS share;
"""
df = neo4j_to_pandas(query)

  Query Results:
-------------------------
(242, 6)


,rel_id,source,target,is_source_energy_project,status,share
0,21086,EP8466,E100000003859,1,announced,100.0
1,38582,E100000003859,E100000000769,0,Entity,0.0
2,35554,E100000000769,E100001000085,0,Entity,100.0
3,38581,E100000003859,E100000000919,0,Entity,0.0
4,38583,E100000003859,E100000001099,0,Entity,0.0


In [8]:
df.status = df.status.apply(lambda x: x.lower() if pd.notnull(x) else x)
df.index.name = 'index'
df.to_json('/user/projects/project-3-energy-ownership/code/Data/data_viz/test/Nigeria_relationships_bu.json', orient='records')

# Top Down
- Blackrock [Complete: add algorithm scores]

In [9]:
# returns entire graph in neo4j
# MATCH path = (startingpoint:Entity {name: 'Blackrock'})<-[:SHARE*]-(m:Entity|Energy_Project)
# WITH path, nodes(path) AS np
# UNWIND np AS node
# RETURN DISTINCT node.name AS name,
#                 node.id AS id,
#                 node.country AS country,
#                 labels(node) AS type,
#                 path
# ORDER BY name;

In [10]:
# grab nodes
query = """
MATCH path = (startingpoint:Entity {name: 'Blackrock'})<-[:SHARE*]-(m:Entity|Energy_Project)
WITH nodes(path) AS np
UNWIND np AS n
RETURN DISTINCT n.name AS name,
                n.id AS id,
                CASE WHEN n:Energy_Project THEN 1 ELSE 0 END AS is_energy_project,
                n.country AS country,
                n.entity_type AS entity_type,
                CASE WHEN n:Energy_Project THEN n.energy_project_type
                     WHEN n:Entity THEN 'Entity'
                     ELSE 'Unknown' END AS energy_project_type,
                n.legal_entity_type AS legal_entity_type,
                n.energy_project_type AS type,
                CASE WHEN n:Energy_Project THEN n.status
                    WHEN n:Entity THEN 'Entity'
                    ELSE 'Unknown' END AS status,
                COALESCE(n.publicly_listed, 'Unknown') AS publicly_listed,
                COALESCE(n.bio_plant_capacity, 0) AS bio_plant_capacity,
                COALESCE(n.coal_mine_capacity, 0) AS coal_mine_capacity,
                COALESCE(n.coal_plant_capacity, 0) AS coal_plant_capacity,
                COALESCE(n.gas_plant_capacity, 0) AS gas_plant_capacity,
                COALESCE(n.steel_plant_capacity, 0) AS steel_plant_capacity,
                COALESCE(n.capacity_mtpa, 0) AS capacity_mtpa,
                COALESCE(n.capacity_mw, 0) AS capacity_mw,
                COALESCE(n.capacity_nominal_crude_steel_capacity_ttpa, 0) AS capacity_nominal_crude_steel_capacity_ttpa,
                apoc.node.degree(n, '<SHARE') AS incoming_share_count
"""
df = neo4j_to_pandas(query)

  Query Results:
-------------------------
(1990, 19)


,name,id,is_energy_project,country,entity_type,energy_project_type,legal_entity_type,type,status,publicly_listed,bio_plant_capacity,coal_mine_capacity,coal_plant_capacity,gas_plant_capacity,steel_plant_capacity,capacity_mtpa,capacity_mw,capacity_nominal_crude_steel_capacity_ttpa,incoming_share_count
0,Blackrock,E100001000348,0,United States,legal entity,Entity,Inc,None,Entity,True,7124.652324413398,1132.5847865058342,161938.89144139216,299859.5469824637,1132.5847865058342,0.0,0.0,0,57
1,BASF,E100000000395,0,Germany,legal entity,Entity,SE,None,Entity,True,0.0,0.0,0.0,317.0435268,0.0,0.0,0.0,0,7
2,"Ludwigshafen Mitte power station Blocks 10, 11...",EP6265,1,Germany,None,Gas or Oil Plant,None,Gas or Oil Plant,operating,Unknown,0,0,0,0,0,0.0,503.0,0,0
3,Ludwigshafen Süd power station 1,EP6266,1,Germany,None,Gas or Oil Plant,None,Gas or Oil Plant,operating,Unknown,0,0,0,0,0,0.0,415.0,0,0
4,Ludwigshafen Mitte power station Block 1,EP6264,1,Germany,None,Gas or Oil Plant,None,Gas or Oil Plant,operating,Unknown,0,0,0,0,0,0.0,94.0,0,0


In [11]:
df.status = df.status.apply(lambda x: x.lower() if pd.notnull(x) else x)
df.index.name = 'index'
df.to_json('/user/projects/project-3-energy-ownership/code/Data/data_viz/test/blackrock_nodes_7_31_24.json', orient='records')

In [12]:
# grab relationships          
query = """
MATCH path = (startingpoint:Entity {name: 'Blackrock'})<-[:SHARE*]-(m:Entity|Energy_Project)
WITH relationships(path) AS rp
UNWIND rp AS rel
RETURN DISTINCT id(rel) AS rel_id,
                startNode(rel).id AS source,
                endNode(rel).id AS target,
                CASE WHEN startNode(rel):Energy_Project THEN 1
                    ELSE 0 END AS is_source_energy_project,
                CASE WHEN startNode(rel):Energy_Project THEN startNode(rel).energy_project_type
                     WHEN startNode(rel):Entity THEN 'Entity'
                     ELSE 'Unknown' END AS energy_project_type,
                CASE WHEN startNode(rel):Entity THEN 'Entity'
                    WHEN startNode(rel):Energy_Project THEN startNode(rel).status
                    ELSE 'Unknown' END AS status,
                rel.share AS share;
"""
df = neo4j_to_pandas(query)

  Query Results:
-------------------------
(2027, 7)


,rel_id,source,target,is_source_energy_project,energy_project_type,status,share
0,35668,E100000000395,E100001000348,0,Entity,Entity,5.14
1,19149,EP6265,E100000000395,1,Gas or Oil Plant,operating,100.00
2,19150,EP6266,E100000000395,1,Gas or Oil Plant,operating,100.00
3,19148,EP6264,E100000000395,1,Gas or Oil Plant,operating,100.00
4,38810,E100000002491,E100000000395,0,Entity,Entity,100.00


In [13]:
df.status = df.status.apply(lambda x: x.lower() if pd.notnull(x) else x)
df.index.name = 'index'
df.to_json('/user/projects/project-3-energy-ownership/code/Data/data_viz/test/blackrock_relationships_7_31_24.json', orient='records')

# All Nodes and Relationships

In [90]:
# return all entity nodes
query = """
MATCH (n:Entity)
RETURN n.name AS name,
    n.id AS id,
    CASE WHEN n:Energy_Project THEN 1
        ELSE 0 END AS is_energy_project,
    n.country AS country,
    n.entity_type AS entity_type,
    CASE WHEN n:Energy_Project THEN n.energy_project_type
        WHEN n:Entity THEN 'Entity'
        ELSE 'Unknown' END AS energy_project_type,
    n.legal_entity_type AS legal_entity_type,
    n.energy_project_type AS type,
    CASE WHEN n:Energy_Project THEN n.status
        WHEN n:Entity THEN 'Entity'
        ELSE 'Unknown' END AS status,
    COALESCE(n.publicly_listed, 'Unknown') AS publicly_listed,
    COALESCE(n.bio_plant_capacity, 0) AS bio_plant_capacity,
    COALESCE(n.coal_mine_capacity, 0) AS coal_mine_capacity,
    COALESCE(n.coal_plant_capacity, 0) AS coal_plant_capacity,
    COALESCE(n.gas_plant_capacity, 0) AS gas_plant_capacity,
    COALESCE(n.steel_plant_capacity, 0) AS steel_plant_capacity,
    COALESCE(n.capacity_mtpa, 0) AS capacity_mtpa,
    COALESCE(n.capacity_mw, 0) AS capacity_mw,
    COALESCE(n.capacity_nominal_crude_steel_capacity_ttpa, 0) AS capacity_nominal_crude_steel_capacity_ttpa,
    apoc.node.degree(n, '<SHARE') AS incoming_share_count
ORDER BY name;
"""
df = neo4j_to_pandas(query)

  Query Results:
-------------------------
(12520, 19)


,name,id,is_energy_project,country,entity_type,energy_project_type,legal_entity_type,type,status,publicly_listed,bio_plant_capacity,coal_mine_capacity,coal_plant_capacity,gas_plant_capacity,steel_plant_capacity,capacity_mtpa,capacity_mw,capacity_nominal_crude_steel_capacity_ttpa,incoming_share_count
0,(Hong Kong) Dewei Industrial Investment,E100000001514,0,Unknown,legal entity,Entity,Co Ltd,None,Entity,Unknown,0.0,0.0,0.0,3528.0000000000005,0.0,0,0,0,1
1,100 Thuwanon,E100000131203,0,Unknown,Unknown,Entity,Co,None,Entity,Unknown,0.0,0.0,0.0,0.0,0.0,0,0,0,1
2,1590 Energy,E100000002359,0,Unknown,legal entity,Entity,Co,None,Entity,Unknown,0.0,0.0,0.0,23520.0,0.0,0,0,0,1
3,1832 Asset Management,E100001014315,0,Canada,legal entity,Entity,LP,None,Entity,False,0.0,0.0,0.0,434.112,0.0,0,0,0,1
4,1947 Trust,E100001016594,0,Singapore,legal entity,Entity,Unknown,None,Entity,Unknown,0.0,10.5240237282411,799.2727066333332,0.0,10.5240237282411,0,0,0,1


In [15]:
df.status = df.status.apply(lambda x: x.lower() if pd.notnull(x) else x)
df.index.name = 'index'
df.to_json('/user/projects/project-3-energy-ownership/code/Data/data_viz/test/all_nodes_7_31_24.json', orient='records')

In [93]:
# return all entity to entity relationships
query = """
MATCH (start:Entity)-[r:SHARE*]->(end:Entity)
UNWIND r AS rel
RETURN DISTINCT id(rel) AS id,
    startNode(rel).id AS source,
    endNode(rel).id AS target,
    CASE WHEN start:Energy_Project THEN 1
        ELSE 0 END AS is_source_energy_project,
    CASE WHEN start:Energy_Project THEN start.energy_project_type
         WHEN start:Entity THEN 'Entity'
         ELSE 'Unknown' END AS energy_project_type,
    CASE WHEN start:Entity THEN 'Entity'
        WHEN start:Energy_Project THEN start.status
        ELSE 'Unknown' END AS status,
    rel.share AS share 
"""

df = neo4j_to_pandas(query)

  Query Results:
-------------------------
(12384, 7)


,id,source,target,is_source_energy_project,energy_project_type,status,share
0,4762,E100000002359,E100000000817,0,Entity,Entity,100.00
1,8050,E100000002360,E100000000689,0,Entity,Entity,100.00
2,4201,E100000000689,E100001010042,0,Entity,Entity,40.00
3,11000,E100001010780,E100000000651,0,Entity,Entity,50.00
4,4231,E100000000651,E100001000179,0,Entity,Entity,5.73


In [18]:
df.status = df.status.apply(lambda x: x.lower() if pd.notnull(x) else x)
df.index.name = 'index'
df.to_json('/user/projects/project-3-energy-ownership/code/Data/data_viz/test/all_relationships_7_31_24.json', orient='records')

In [207]:
# return all relationships
query = """
MATCH (start:Energy_Project)-[r:SHARE*]->(end:Entity)
UNWIND r AS rel
RETURN DISTINCT id(rel) AS id,
    startNode(rel).id AS source,
    endNode(rel).id AS target,
    CASE WHEN start:Energy_Project THEN 1
        ELSE 0 END AS is_source_energy_project,
    CASE WHEN start:Energy_Project THEN start.energy_project_type
         WHEN start:Entity THEN 'Entity'
         ELSE 'Unknown' END AS energy_project_type,
    CASE WHEN start:Entity THEN 'Entity'
        WHEN start:Energy_Project THEN start.status
        ELSE 'Unknown' END AS status,
    rel.share AS share 
"""

df = neo4j_to_pandas(query)

  Query Results:
-------------------------
(40174, 7)


,id,source,target,is_source_energy_project,energy_project_type,status,share
0,33900,EP2,E100000001858,1,Gas or Oil Plant,operating,100.0
1,4050,E100000001858,E100000000354,1,Gas or Oil Plant,operating,100.0
2,4051,E100000000354,E100001000146,1,Gas or Oil Plant,operating,100.0
3,33901,EP3,E100000001858,1,Gas or Oil Plant,operating,100.0
4,33902,EP4,E100000001858,1,Gas or Oil Plant,operating,100.0


In [208]:
df[df.source == 'EP23137']

,id,source,target,is_source_energy_project,energy_project_type,status,share


# Creating Subgraphs For Algorithms

- You only need to create the subgraphs once!

In [120]:
# # run if you want to drop and recreate the subgraphs
# query = "CALL gds.graph.drop('ds_graph', false)"
# session.run(query)

# query = "CALL gds.graph.drop('coal_plant_operating_subgraph_final', false)"
# session.run(query)

# query = "CALL gds.graph.drop('steel_plant_operating_subgraph_final', false)"
# session.run(query)

# query = "CALL gds.graph.drop('gas_oil_plant_operating_subgraph_final', false)"
# session.run(query)

# query = "CALL gds.graph.drop('coal_mine_operating_subgraph_final', false)"
# session.run(query)

# query = "CALL gds.graph.drop('bio_plant_operating_subgraph_final', false)"
# session.run(query)

In [121]:
# entire graph
query = "CALL gds.graph.project('ds_graph', 'Entity', 'SHARE', {relationshipProperties: 'share'})"
session.run(query)

In [122]:
# coal plant final!
query = """

CALL gds.graph.project.cypher(
  'coal_plant_operating_subgraph_final',
  'MATCH (ep:Energy_Project)
   WHERE ep.energy_project_type = "Coal Plant" AND ep.status IN ["operating", "operating pre-retirement"]
   RETURN DISTINCT id(ep) AS id

   UNION

   MATCH (ep:Energy_Project)-[r:SHARE*]->(e:Entity)
   WHERE ep.energy_project_type = "Coal Plant" AND ep.status IN ["operating", "operating pre-retirement"]
   UNWIND r AS rel
   RETURN DISTINCT id(endNode(rel)) AS id',
   
  'MATCH (ep:Energy_Project)-[r:SHARE*]->(e:Entity)
   WHERE ep.energy_project_type = "Coal Plant" AND ep.status IN ["operating", "operating pre-retirement"]
   UNWIND r AS rel
   RETURN DISTINCT id(rel) AS id, id(startNode(rel)) AS source, id(endNode(rel)) AS target, rel.share AS share'
)
YIELD graphName AS graph, nodeCount AS nodes, relationshipCount AS rels
RETURN graph, nodes, rels;

"""

my_neo4j_run_query_pandas(query)

,graph,nodes,rels
0,coal_plant_operating_subgraph_final,11594,11976


In [12]:
# steel plant subgraph
query = """

CALL gds.graph.project.cypher(
  'steel_plant_operating_subgraph_final',
  'MATCH (ep:Energy_Project)
   WHERE ep.energy_project_type = "Steel Plant" AND ep.status IN ["operating", "operating pre-retirement"]
   RETURN DISTINCT id(ep) AS id

   UNION

   MATCH (ep:Energy_Project)-[r:SHARE*]->(e:Entity)
   WHERE ep.energy_project_type = "Steel Plant" AND ep.status IN ["operating", "operating pre-retirement"]
   UNWIND r AS rel
   RETURN DISTINCT id(endNode(rel)) AS id',
   
  'MATCH (ep:Energy_Project)-[r:SHARE*]->(e:Entity)
   WHERE ep.energy_project_type = "Steel Plant" AND ep.status IN ["operating", "operating pre-retirement"]
   UNWIND r AS rel
   RETURN DISTINCT id(rel) AS id, id(startNode(rel)) AS source, id(endNode(rel)) AS target, rel.share AS share'
)
YIELD graphName AS graph, nodeCount AS nodes, relationshipCount AS rels
RETURN graph, nodes, rels;

"""

my_neo4j_run_query_pandas(query)

,graph,nodes,rels
0,steel_plant_operating_subgraph_final,2872,2616


In [124]:
# gas oil plant subgraph
query = """

CALL gds.graph.project.cypher(
  'gas_oil_plant_operating_subgraph_final',
  'MATCH (ep:Energy_Project)
   WHERE ep.energy_project_type = "Gas or Oil Plant" AND ep.status IN ["operating", "operating pre-retirement"]
   RETURN DISTINCT id(ep) AS id

   UNION

   MATCH (ep:Energy_Project)-[r:SHARE*]->(e:Entity)
   WHERE ep.energy_project_type = "Gas or Oil Plant" AND ep.status IN ["operating", "operating pre-retirement"]
   UNWIND r AS rel
   RETURN DISTINCT id(endNode(rel)) AS id',
   
  'MATCH (ep:Energy_Project)-[r:SHARE*]->(e:Entity)
   WHERE ep.energy_project_type = "Gas or Oil Plant" AND ep.status IN ["operating", "operating pre-retirement"]
   UNWIND r AS rel
   RETURN DISTINCT id(rel) AS id, id(startNode(rel)) AS source, id(endNode(rel)) AS target, rel.share AS share'
)
YIELD graphName AS graph, nodeCount AS nodes, relationshipCount AS rels
RETURN graph, nodes, rels;

"""

my_neo4j_run_query_pandas(query)

,graph,nodes,rels
0,gas_oil_plant_operating_subgraph_final,14781,13691


In [125]:
# coal mine
query = """

CALL gds.graph.project.cypher(
  'coal_mine_operating_subgraph_final',
  'MATCH (ep:Energy_Project)
   WHERE ep.energy_project_type = "Coal Mine" AND ep.status IN ["Operating", "Operating pre-retirement"]
   RETURN DISTINCT id(ep) AS id

   UNION

   MATCH (ep:Energy_Project)-[r:SHARE*]->(e:Entity)
   WHERE ep.energy_project_type = "Coal Mine" AND ep.status IN ["Operating", "Operating pre-retirement"]
   UNWIND r AS rel
   RETURN DISTINCT id(endNode(rel)) AS id',
   
  'MATCH (ep:Energy_Project)-[r:SHARE*]->(e:Entity)
   WHERE ep.energy_project_type = "Coal Mine" AND ep.status IN ["Operating", "Operating pre-retirement"]
   UNWIND r AS rel
   RETURN DISTINCT id(rel) AS id, id(startNode(rel)) AS source, id(endNode(rel)) AS target, rel.share AS share'
)
YIELD graphName AS graph, nodeCount AS nodes, relationshipCount AS rels
RETURN graph, nodes, rels;
"""

my_neo4j_run_query_pandas(query)

,graph,nodes,rels
0,coal_mine_operating_subgraph_final,3423,2539


In [ ]:
# not implemented;
# query = """

# CALL gds.graph.project.cypher(
#   'bio_plant_operating_subgraph_final',
#   'MATCH (n:Energy_Project)
#    WHERE n.energy_project_type = "Bioenergy Plant" AND (n.status = "operating" OR n.status = "operating pre-retirement")
#    RETURN id(n) AS id
#    UNION
#    MATCH (n:Entity)
#    RETURN id(n) AS id',
#   'MATCH (n1:Energy_Project)-[r:SHARE]->(n2:Entity)
#    WHERE n1.energy_project_type = "Bioenergy Plant" AND (n1.status = "operating" OR n1.status = "operating pre-retirement")
#    RETURN id(n1) AS source, id(n2) AS target'
# )
# YIELD graphName AS graph, nodeCount AS nodes, relationshipCount AS rels
# RETURN graph, nodes, rels;
# """

# session.run(query)

## Harmonic Centrality / Closeness
- These queries also update the node values with a new attribute via the SET command

In [14]:
entity_df = pd.read_csv('/user/projects/project-3-energy-ownership/code/Data/entity_nodes.csv')

In [127]:
# entire graph
query = """

CALL gds.alpha.closeness.harmonic.stream('ds_graph', {})
YIELD nodeId, centrality
MATCH (n) WHERE id(n) = nodeId
SET n.centrality = centrality
RETURN n.id AS id, n.name AS name, n.centrality AS centrality
ORDER BY n.centrality DESC;

"""

centrality_df = my_neo4j_run_query_pandas(query)
pd.merge(centrality_df, entity_df[['ID', 'Country']], how='left', left_on='id', right_on='ID').head(10)

,id,name,centrality,ID,Country
0,E100001000348,Blackrock,0.017444,E100001000348,United States
1,E100001010595,The Vanguard Group,0.015932,E100001010595,United States
2,E100001015833,Blackrock Advisors,0.012139,E100001015833,NaN
3,E100001000178,The Master Trust Bank of Japan,0.010784,E100001000178,Japan
4,E100001000179,Custody Bank of Japan,0.009399,E100001000179,Japan
5,E100000000239,China Energy Investment,0.008743,E100000000239,China
6,E100001000382,Nippon Life Insurance,0.007156,E100001000382,Japan
7,E100001011617,Meiji Yasuda Life Insurance,0.006956,E100001011617,Japan
8,E100001016125,Mitsubishi UFJ Trust and Banking,0.006916,E100001016125,Japan
9,E100001016126,The Norinchukin Trust & Banking,0.006916,E100001016126,Japan


In [159]:
# coal plant
query = """

CALL gds.alpha.closeness.harmonic.stream('coal_plant_operating_subgraph_final', {})
YIELD nodeId, centrality
MATCH (n) WHERE id(n) = nodeId
SET n.centrality_coal_plant = centrality
RETURN n.id AS id, n.name AS name, n.centrality_coal_plant AS centrality_coal_plant
ORDER BY n.centrality_coal_plant DESC;

"""

centrality_df = my_neo4j_run_query_pandas(query)
pd.merge(centrality_df, entity_df[['ID', 'Country']], how='left', left_on='id', right_on='ID').head(10)

,id,name,centrality_coal_plant,ID,Country
0,E100000000239,China Energy Investment,0.019726,E100000000239,China
1,E100000000217,Huaneng Power International,0.014305,E100000000217,China
2,E100001000397,Government of India,0.013997,E100001000397,India
3,E100001000520,Life Insurance Corporation of India,0.012551,E100001000520,India
4,E100001000348,Blackrock,0.012331,E100001000348,United States
5,E100000001926,NTPC,0.012033,E100000001926,India
6,E100000000175,China Shenhua Energy,0.011652,E100000000175,China
7,E100001010595,The Vanguard Group,0.011145,E100001010595,United States
8,E100001000178,The Master Trust Bank of Japan,0.009041,E100001000178,Japan
9,E100001015833,Blackrock Advisors,0.008887,E100001015833,NaN


In [15]:
# steel plant
query = """

CALL gds.alpha.closeness.harmonic.stream('steel_plant_operating_subgraph_final', {})
YIELD nodeId, centrality
MATCH (n) WHERE id(n) = nodeId
SET n.centrality_steel_plant = centrality
RETURN n.id AS id, n.name AS name, n.centrality_steel_plant AS centrality_steel_plant
ORDER BY n.centrality_steel_plant DESC;

"""

centrality_df = my_neo4j_run_query_pandas(query)
pd.merge(centrality_df, entity_df[['ID', 'Country']], how='left', left_on='id', right_on='ID').head(10)

,id,name,centrality_steel_plant,ID,Country
0,E100001000348,Blackrock,0.026303,E100001000348,United States
1,E100001010595,The Vanguard Group,0.023795,E100001010595,United States
2,E100001015833,Blackrock Advisors,0.018603,E100001015833,NaN
3,E100000000687,ArcelorMittal,0.016777,E100000000687,Luxembourg
4,E100001000178,The Master Trust Bank of Japan,0.014948,E100001000178,Japan
5,E100001000179,Custody Bank of Japan,0.013294,E100001000179,Japan
6,E100000000652,Nippon Steel,0.011901,E100000000652,Japan
7,E100001016126,The Norinchukin Trust & Banking,0.009915,E100001016126,Japan
8,E100001011617,Meiji Yasuda Life Insurance,0.009915,E100001011617,Japan
9,E100001016125,Mitsubishi UFJ Trust and Banking,0.009915,E100001016125,Japan


In [161]:
# gas oil plant subgraph
query = """

CALL gds.alpha.closeness.harmonic.stream('gas_oil_plant_operating_subgraph_final', {})
YIELD nodeId, centrality
MATCH (n) WHERE id(n) = nodeId
SET n.centrality_gas_and_oil_plant = centrality
RETURN n.id AS id, n.name AS name, n.centrality_gas_and_oil_plant AS centrality_gas_and_oil_plant
ORDER BY n.centrality_gas_and_oil_plant DESC;

"""

centrality_df = my_neo4j_run_query_pandas(query)
pd.merge(centrality_df, entity_df[['ID', 'Country']], how='left', left_on='id', right_on='ID').head(10)

,id,name,centrality_gas_and_oil_plant,ID,Country
0,E100000000890,Saudi Electricity,0.030582,E100000000890,Saudi Arabia
1,E100001000348,Blackrock,0.026820,E100001000348,United States
2,E100001010595,The Vanguard Group,0.024730,E100001010595,United States
3,E100001015833,Blackrock Advisors,0.020135,E100001015833,NaN
4,E100000001358,Public Investment Fund (Saudi Arabia),0.017705,E100000001358,NaN
5,E100001000178,The Master Trust Bank of Japan,0.015532,E100001000178,Japan
6,E100000001354,Ministry of Electricity (Iraq),0.015494,E100000001354,NaN
7,E100001000322,Saudi Aramco Power,0.015415,E100001000322,Saudi Arabia
8,E100001000198,Government of Saudi Arabia,0.015222,E100001000198,Saudi Arabia
9,E100000000888,Saudi Arabian Oil,0.013109,E100000000888,Saudi Arabia


In [162]:
# coal mine subgraph
query = """

CALL gds.alpha.closeness.harmonic.stream('coal_mine_operating_subgraph_final', {})
YIELD nodeId, centrality
MATCH (n) WHERE id(n) = nodeId
SET n.centrality_coal_mine = centrality
RETURN n.id AS id, n.name AS name, n.centrality_coal_mine AS centrality_coal_mine
ORDER BY n.centrality_coal_mine DESC;

"""

centrality_df = my_neo4j_run_query_pandas(query)
pd.merge(centrality_df, entity_df[['ID', 'Country']], how='left', left_on='id', right_on='ID').head(10)

,id,name,centrality_coal_mine,ID,Country
0,E100001000397,Government of India,0.041214,E100001000397,India
1,E100001000396,Coal India,0.040766,E100001000396,India
2,E100001000520,Life Insurance Corporation of India,0.034458,E100001000520,India
3,E100001000348,Blackrock,0.024528,E100001000348,United States
4,E100001010595,The Vanguard Group,0.022831,E100001010595,United States
5,E100001000804,Eastern Coalfields,0.020164,E100001000804,India
6,E100001015833,Blackrock Advisors,0.018031,E100001015833,NaN
7,E100001000806,South Eastern Coalfields,0.017241,E100001000806,India
8,E100001000801,Western Coalfields,0.011689,E100001000801,India
9,E100001000809,The Singareni Collieries,0.011689,E100001000809,India


## Betweenness Centrality
- only ran for coal mine / coal plant

In [163]:
# all nodes
query = """

CALL gds.betweenness.stream('ds_graph', { relationshipWeightProperty: 'share' })
YIELD nodeId, score
MATCH (n) WHERE id(n) = nodeId
SET n.betweenness = score
RETURN n.id AS id, n.name AS name, n.betweenness AS betweenness
ORDER BY betweenness DESC;

"""

betweenness_df = my_neo4j_run_query_pandas(query)
pd.merge(betweenness_df, entity_df[['ID', 'Country']], how='left', left_on='id', right_on='ID').head(10)

,id,name,betweenness,ID,Country
0,E100001000178,The Master Trust Bank of Japan,1525.0,E100001000178,Japan
1,E100001000179,Custody Bank of Japan,1174.0,E100001000179,Japan
2,E100000123438,Chongqing Energy Investment Group,809.0,E100000123438,China
3,E100001000348,Blackrock,655.0,E100001000348,United States
4,E100000000651,Mitsui & Co,484.0,E100000000651,Japan
5,E100000001533,JERA,362.0,E100000001533,Japan
6,E100000000649,Marubeni,338.0,E100000000649,Japan
7,E100001000400,Glencore,336.0,E100001000400,Jersey
8,E100000124937,Chongqing Energy Investment Runxin No. 1 Enter...,290.0,E100000124937,China
9,E100000000650,Mitsubishi,283.0,E100000000650,Japan


In [164]:
# coal plant
query = """

CALL gds.betweenness.stream('coal_plant_operating_subgraph_final', { relationshipWeightProperty: 'share' })
YIELD nodeId, score
MATCH (n) WHERE id(n) = nodeId
SET n.betweenness_coal_plant = score
RETURN n.id AS id, n.name AS name, n.betweenness_coal_plant AS betweenness
ORDER BY betweenness DESC;

"""

betweenness_df = my_neo4j_run_query_pandas(query)
pd.merge(betweenness_df, entity_df[['ID', 'Country']], how='left', left_on='id', right_on='ID').head(10)

,id,name,betweenness,ID,Country
0,E100000123438,Chongqing Energy Investment Group,1800.0,E100000123438,China
1,E100001000178,The Master Trust Bank of Japan,1408.0,E100001000178,Japan
2,E100001000179,Custody Bank of Japan,954.0,E100001000179,Japan
3,E100000001471,En+ Group,792.0,E100000001471,Russia
4,E100001010442,Glencore Group Funding,712.0,E100001010442,United Arab Emirates
5,E100000001533,JERA,690.0,E100000001533,Japan
6,E100001000400,Glencore,651.0,E100001000400,Jersey
7,E100001000348,Blackrock,609.0,E100001000348,United States
8,E100000124937,Chongqing Energy Investment Runxin No. 1 Enter...,609.0,E100000124937,China
9,E100001000404,Siberian Coal Energy Company,524.0,E100001000404,Russia


In [16]:
# steel plan
query = """

CALL gds.betweenness.stream('steel_plant_operating_subgraph_final', { relationshipWeightProperty: 'share' })
YIELD nodeId, score
MATCH (n) WHERE id(n) = nodeId
SET n.betweenness_steel_plant = score
RETURN n.id AS id, n.name AS name, n.betweenness_steel_plant AS betweenness
ORDER BY betweenness DESC;

"""

betweenness_df = my_neo4j_run_query_pandas(query)
pd.merge(betweenness_df, entity_df[['ID', 'Country']], how='left', left_on='id', right_on='ID').head(10)

,id,name,betweenness,ID,Country
0,E100001000178,The Master Trust Bank of Japan,516.0,E100001000178,Japan
1,E100001000179,Custody Bank of Japan,424.0,E100001000179,Japan
2,E100000000652,Nippon Steel,334.0,E100000000652,Japan
3,E100001000348,Blackrock,246.0,E100001000348,United States
4,E100000124495,Tianjin Jianlong Iron & Steel Industrial,228.0,E100000124495,China
5,E100000000687,ArcelorMittal,225.0,E100000000687,Luxembourg
6,E100000000651,Mitsui & Co,210.0,E100000000651,Japan
7,E100000000643,JFE Holdings,195.0,E100000000643,Japan
8,E100001000516,JSW Steel,156.0,E100001000516,India
9,E100000003335,JFE Steel,154.0,E100000003335,Japan


In [165]:
# coal mine
query = """

CALL gds.betweenness.stream('coal_mine_operating_subgraph_final', { relationshipWeightProperty: 'share' })
YIELD nodeId, score
MATCH (n) WHERE id(n) = nodeId
SET n.betweenness_coal_mine = score
RETURN n.id AS id, n.name AS name, n.betweenness_coal_mine AS betweenness
ORDER BY betweenness DESC;

"""

betweenness_df = my_neo4j_run_query_pandas(query)
pd.merge(betweenness_df, entity_df[['ID', 'Country']], how='left', left_on='id', right_on='ID').head(10)

,id,name,betweenness,ID,Country
0,E100001000396,Coal India,539.0,E100001000396,India
1,E100001000400,Glencore,469.0,E100001000400,Jersey
2,E100001000178,The Master Trust Bank of Japan,459.0,E100001000178,Japan
3,E100001000179,Custody Bank of Japan,374.0,E100001000179,Japan
4,E100001000348,Blackrock,317.0,E100001000348,United States
5,E100001000804,Eastern Coalfields,207.0,E100001000804,India
6,E100000001471,En+ Group,189.0,E100000001471,Russia
7,E100000131246,Alpha Metallurgical Resources,186.0,E100000131246,United States
8,E100001010442,Glencore Group Funding,176.0,E100001010442,United Arab Emirates
9,E100001000806,South Eastern Coalfields,175.5,E100001000806,India


# Page Rank

In [141]:
query = """

CALL gds.pageRank.stream('ds_graph',
                         { maxIterations: $max_iterations,
                           dampingFactor: $damping_factor}
                         )
YIELD nodeId, score
SET gds.util.asNode(nodeId).page_rank = score
RETURN gds.util.asNode(nodeId).id AS id, gds.util.asNode(nodeId).name AS name, score as page_rank
ORDER BY page_rank DESC, name ASC

"""

max_iterations = 20
damping_factor = 0.05

pagerank_df = my_neo4j_run_query_pandas(query, max_iterations=max_iterations, damping_factor=damping_factor)
pd.merge(pagerank_df, entity_df[['ID', 'Country']], how='left', left_on='id', right_on='ID').head(10)

,id,name,page_rank,ID,Country
0,E100000000217,Huaneng Power International,2.981387,E100000000217,China
1,E100000000213,Huadian Power International,2.544290,E100000000213,China
2,E100000001170,Calpine,2.493750,E100000001170,United States
3,E100001000348,Blackrock,2.316280,E100001000348,United States
4,E100000000239,China Energy Investment,2.272009,E100000000239,China
5,E100000001322,Vistra,2.187375,E100000001322,United States
6,E100000000181,Datang International Power Generation,2.182229,E100000000181,China
7,E100000001262,NRG Energy,1.940177,E100000001262,United States
8,E100000122607,China Resources Power Investment,1.919792,E100000122607,China
9,E100000000687,ArcelorMittal,1.892519,E100000000687,Luxembourg


In [18]:
# steel plant
query = """

CALL gds.pageRank.stream('steel_plant_operating_subgraph_final',
                         { maxIterations: $max_iterations,
                           dampingFactor: $damping_factor}
                         )
YIELD nodeId, score
SET gds.util.asNode(nodeId).page_rank_steel_plant = score
RETURN gds.util.asNode(nodeId).id AS id, gds.util.asNode(nodeId).name AS name, score as page_rank
ORDER BY page_rank DESC, name ASC

"""

max_iterations = 20
damping_factor = 0.05

pagerank_df = my_neo4j_run_query_pandas(query, max_iterations=max_iterations, damping_factor=damping_factor)
pd.merge(pagerank_df, entity_df[['ID', 'Country']], how='left', left_on='id', right_on='ID').head(10)

,id,name,page_rank,ID,Country
0,E100000000687,ArcelorMittal,2.195157,E100000000687,Luxembourg
1,E100001010181,Nucor,1.894062,E100001010181,United States
2,E100000000652,Nippon Steel,1.764566,E100000000652,Japan
3,E100001000649,Cleveland-Cliffs,1.520000,E100001000649,United States
4,E100000130950,Liberty Steel Group,1.387000,E100000130950,United Kingdom
5,E100001000516,JSW Steel,1.384625,E100001000516,India
6,E100001000703,Steel Authority of India,1.377500,E100001000703,India
7,E100001010018,Tata Steel,1.368665,E100001010018,India
8,E100000131000,Riva Forni Elettrici,1.353987,E100000131000,Italy
9,E100000130754,Commercial Metals,1.332375,E100000130754,United States


In [142]:
# coal plant
query = """

CALL gds.pageRank.stream('coal_plant_operating_subgraph_final',
                         { maxIterations: $max_iterations,
                           dampingFactor: $damping_factor}
                         )
YIELD nodeId, score
SET gds.util.asNode(nodeId).page_rank_coal_plant = score
RETURN gds.util.asNode(nodeId).id AS id, gds.util.asNode(nodeId).name AS name, score as page_rank
ORDER BY page_rank DESC, name ASC

"""

max_iterations = 20
damping_factor = 0.05

pagerank_df = my_neo4j_run_query_pandas(query, max_iterations=max_iterations, damping_factor=damping_factor)
pd.merge(pagerank_df, entity_df[['ID', 'Country']], how='left', left_on='id', right_on='ID').head(10)

,id,name,page_rank,ID,Country
0,E100000001926,NTPC,6.752521,E100000001926,India
1,E100000000921,Eskom Holdings SOC,4.797500,E100000000921,South Africa
2,E100000000217,Huaneng Power International,3.194485,E100000000217,China
3,E100000120122,Baikal Energy Company,2.850000,E100000120122,Russia
4,E100000000181,Datang International Power Generation,2.642544,E100000000181,China
5,E100000000537,PT PLN (Persero),2.503280,E100000000537,Indonesia
6,E100000003018,Far Eastern Generating Company,2.327500,E100000003018,Russia
7,E100000120560,Hindalco Industries,2.327500,E100000120560,India
8,E100000185999,PT PLN (Persero),2.256250,E100000185999,NaN
9,E100000001923,Maharashtra State Power Generation,2.232500,E100000001923,India


In [143]:
# coal mine
query = """

CALL gds.pageRank.stream('coal_mine_operating_subgraph_final',
                         { maxIterations: $max_iterations,
                           dampingFactor: $damping_factor}
                         )
YIELD nodeId, score
SET gds.util.asNode(nodeId).page_rank_coal_mine = score
RETURN gds.util.asNode(nodeId).id AS id, gds.util.asNode(nodeId).name AS name, score as page_rank
ORDER BY page_rank DESC, name ASC

"""

max_iterations = 20
damping_factor = 0.05

pagerank_df = my_neo4j_run_query_pandas(query, max_iterations=max_iterations, damping_factor=damping_factor)
pd.merge(pagerank_df, entity_df[['ID', 'Country']], how='left', left_on='id', right_on='ID').head(10)

,id,name,page_rank,ID,Country
0,E100001000804,Eastern Coalfields,4.227500,E100001000804,India
1,E100001000806,South Eastern Coalfields,3.728750,E100001000806,India
2,E100001000809,The Singareni Collieries,2.850000,E100001000809,India
3,E100001000801,Western Coalfields,2.826250,E100001000801,India
4,E100001000803,Central Coalfields,2.612500,E100001000803,India
5,E100001000805,Bharat Coking Coal,2.327500,E100001000805,India
6,E100001000396,Coal India,1.973625,E100001000396,India
7,E100001000807,Mahanadi Coalfields,1.852500,E100001000807,India
8,E100000132142,Rosebud Mining,1.807375,E100000132142,United States
9,E100001000537,Vietnam National Coal and Mineral Industries H...,1.700500,E100001000537,Vietnam


# Subgraphs for Viz
- coal plant (only entities)

In [186]:
# grab only entities in coal plant network
query = """

MATCH (ep:Energy_Project)-[r:SHARE*]->(e:Entity)
WHERE ep.energy_project_type = "Coal Plant" AND ep.status IN ["operating", "operating pre-retirement"]
UNWIND r AS rel
RETURN DISTINCT id(endNode(rel)) AS neo4j_id,
    endNode(rel).name AS name,
    endNode(rel).id AS id,
    CASE WHEN endNode(rel):Energy_Project THEN 1
        ELSE 0 END AS is_energy_project,
    endNode(rel).country AS country,
    CASE WHEN endNode(rel):Energy_Project THEN endNode(rel).energy_project_type
        WHEN endNode(rel):Entity THEN 'Entity'
        ELSE 'Unknown' END AS energy_project_type,
    CASE WHEN endNode(rel):Energy_Project THEN endNode(rel).status
        WHEN endNode(rel):Entity THEN 'Entity'
        ELSE 'Unknown' END AS status,
    COALESCE(endNode(rel).coal_plant_capacity, 0) AS capacity,
    COALESCE(endNode(rel).centrality_coal_plant, 0) AS centrality,
    COALESCE(endNode(rel).betweenness_coal_plant, 0) AS betweenness,
    COALESCE(endNode(rel).page_rank_coal_plant, 0) AS pagerank,
    apoc.node.degree(endNode(rel), '<SHARE') AS incoming_share_count

"""
df = neo4j_to_pandas(query, query_type='nodes')

  Query Results:
-------------------------
(5069, 12)


,neo4j_id,name,id,is_energy_project,country,energy_project_type,status,capacity,centrality,betweenness,pagerank,incoming_share_count
0,6736,Abakanskaya CHPP,E100000120023,0,Russia,Entity,Entity,38117.39776,0.000345,20.0,1.140000,4
1,16562,Siberian Coal Energy Company,E100001000404,0,Russia,Entity,Entity,1559952.5241554247,0.005722,524.0,1.617612,18
2,6828,Aim Capital,E100001000753,0,Cyprus,Entity,Entity,1559952.5241554247,0.003824,396.0,1.030881,1
3,13055,Linea (CY),E100001000857,0,Cyprus,Entity,Entity,1559952.5241554247,0.002924,266.0,1.001544,1
4,9869,Firstline Trust,E100001016419,0,Unknown,Entity,Entity,1559952.5241554247,0.002394,134.0,1.000077,1


In [187]:
df.status = df.status.apply(lambda x: x.lower() if pd.notnull(x) else x)
df.capacity = df.capacity.astype(float)
df.index.name = 'index'
df.to_json('/user/projects/project-3-energy-ownership/code/Data/data_viz/test/coal_plant_nodes_8_6_24.json', orient='records')

In [181]:
# grab only entity to entity relationships in network
query = """

MATCH (ep:Energy_Project)-[:SHARE]->(start:Entity)-[r:SHARE*]->(end:Entity)
WHERE ep.energy_project_type = "Coal Plant" AND ep.status IN ["operating", "operating pre-retirement"]
UNWIND r AS rel
RETURN DISTINCT id(rel) AS rel_id,
    startNode(rel).id AS source,
    endNode(rel).id AS target,
    CASE WHEN startNode(rel):Energy_Project THEN 1
        ELSE 0 END AS is_source_energy_project,
    CASE WHEN startNode(rel):Energy_Project THEN startNode(rel).energy_project_type
         WHEN startNode(rel):Entity THEN 'Entity'
         ELSE 'Unknown' END AS energy_project_type,
    CASE WHEN startNode(rel):Entity THEN 'Entity'
        WHEN startNode(rel):Energy_Project THEN startNode(rel).status
        ELSE 'Unknown' END AS status,
    rel.share AS share;

"""
df = neo4j_to_pandas(query)

  Query Results:
-------------------------
(5446, 7)


,rel_id,source,target,is_source_energy_project,energy_project_type,status,share
0,8864,E100000120023,E100001000404,0,Entity,Entity,100.00
1,7395,E100001000404,E100001000753,0,Entity,Entity,92.20
2,7393,E100001000753,E100001000857,0,Entity,Entity,99.38
3,7397,E100001000857,E100001016419,0,Entity,Entity,0.00
4,7396,E100001016419,E100001016421,0,Entity,Entity,0.00


In [107]:
df.status = df.status.apply(lambda x: x.lower() if pd.notnull(x) else x)
df.index.name = 'index'
df.to_json('/user/projects/project-3-energy-ownership/code/Data/data_viz/test/coal_plant_relationships_8_6_24.json', orient='records')

- coal plant all entities

In [4]:
# all entities
query = """

MATCH (ep:Energy_Project)
WHERE ep.energy_project_type = "Coal Plant" AND ep.status IN ["operating", "operating pre-retirement"]
RETURN DISTINCT id(ep) AS neo4j_id,
    ep.name AS name,
    ep.id AS id,
    CASE WHEN ep:Energy_Project THEN 1
        ELSE 0 END AS is_energy_project,
    ep.country AS country,
    CASE WHEN ep:Energy_Project THEN ep.energy_project_type
        WHEN ep:Entity THEN 'Entity'
        ELSE 'Unknown' END AS energy_project_type,
    CASE WHEN ep:Energy_Project THEN ep.status
        WHEN ep:Entity THEN 'Entity'
        ELSE 'Unknown' END AS status,
    COALESCE(ep.coal_mine_capacity, 0) AS capacity,
    COALESCE(ep.centrality_coal_mine, 0) AS centrality,
    COALESCE(ep.betweenness_coal_mine, 0) AS betweenness,
    COALESCE(ep.page_rank_coal_mine, 0) AS pagerank,
    apoc.node.degree(ep, '<SHARE') AS incoming_share_count

UNION

MATCH (ep:Energy_Project)-[r:SHARE*]->(e:Entity)
WHERE ep.energy_project_type = "Coal Plant" AND ep.status IN ["operating", "operating pre-retirement"]
UNWIND r AS rel
RETURN DISTINCT id(endNode(rel)) AS neo4j_id,
    endNode(rel).name AS name,
    endNode(rel).id AS id,
    CASE WHEN endNode(rel):Energy_Project THEN 1
        ELSE 0 END AS is_energy_project,
    endNode(rel).country AS country,
    CASE WHEN endNode(rel):Energy_Project THEN endNode(rel).energy_project_type
        WHEN endNode(rel):Entity THEN 'Entity'
        ELSE 'Unknown' END AS energy_project_type,
    CASE WHEN endNode(rel):Energy_Project THEN endNode(rel).status
        WHEN endNode(rel):Entity THEN 'Entity'
        ELSE 'Unknown' END AS status,
    COALESCE(endNode(rel).coal_plant_capacity, 0) AS capacity,
    COALESCE(endNode(rel).centrality_coal_plant, 0) AS centrality,
    COALESCE(endNode(rel).betweenness_coal_plant, 0) AS betweenness,
    COALESCE(endNode(rel).page_rank_coal_plant, 0) AS pagerank,
    apoc.node.degree(endNode(rel), '<SHARE') AS incoming_share_count

"""
df = neo4j_to_pandas(query, query_type='nodes')

  Query Results:
-------------------------
(11594, 12)


,neo4j_id,name,id,is_energy_project,country,energy_project_type,status,capacity,centrality,betweenness,pagerank,incoming_share_count
0,31307,Abakan power station Unit 1,EP12160,1,Russia,Coal Plant,operating,0,0.0,0.0,0.0,0
1,31308,Abakan power station Unit 2,EP12161,1,Russia,Coal Plant,operating,0,0.0,0.0,0.0,0
2,31309,Abakan power station Unit 3,EP12162,1,Russia,Coal Plant,operating,0,0.0,0.0,0.0,0
3,31310,Abakan power station Unit 4,EP12163,1,Russia,Coal Plant,operating,0,0.0,0.0,0.0,0
4,31311,Aboño power station Unit 1,EP12164,1,Spain,Coal Plant,operating,0,0.0,0.0,0.0,0


In [5]:
df.status = df.status.apply(lambda x: x.lower() if pd.notnull(x) else x)
df.capacity = df.capacity.astype(float)
df.index.name = 'index'
df.to_json('/user/projects/project-3-energy-ownership/code/Data/data_viz/test/coal_plant_nodes_all_8_6_24.json', orient='records')

In [7]:
# grab all relationships
query = """

MATCH (ep:Energy_Project)-[r:SHARE*]->(e:Entity)
WHERE ep.energy_project_type = "Coal Plant" AND ep.status IN ["operating", "operating pre-retirement"]
UNWIND r AS rel
RETURN DISTINCT id(rel) AS rel_id,
    startNode(rel).id AS source,
    endNode(rel).id AS target,
    CASE WHEN startNode(rel):Energy_Project THEN 1
        ELSE 0 END AS is_source_energy_project,
    CASE WHEN startNode(rel):Energy_Project THEN startNode(rel).energy_project_type
         WHEN startNode(rel):Entity THEN 'Entity'
         ELSE 'Unknown' END AS energy_project_type,
    CASE WHEN startNode(rel):Entity THEN 'Entity'
        WHEN startNode(rel):Energy_Project THEN startNode(rel).status
        ELSE 'Unknown' END AS status,
    rel.share AS share;

"""
df = neo4j_to_pandas(query)

  Query Results:
-------------------------
(11976, 7)


,rel_id,source,target,is_source_energy_project,energy_project_type,status,share
0,44954,EP12160,E100000120023,1,Coal Plant,operating,100.00
1,8864,E100000120023,E100001000404,0,Entity,Entity,100.00
2,7395,E100001000404,E100001000753,0,Entity,Entity,92.20
3,7393,E100001000753,E100001000857,0,Entity,Entity,99.38
4,7397,E100001000857,E100001016419,0,Entity,Entity,0.00


In [8]:
df.status = df.status.apply(lambda x: x.lower() if pd.notnull(x) else x)
df.index.name = 'index'
df.to_json('/user/projects/project-3-energy-ownership/code/Data/data_viz/test/coal_plant_relationships_all_8_6_24.json', orient='records')

- coal mine

In [214]:
# grab entire subgraph; there are "loose mines". example, can't find 'E100000131796' entity to link for 'EP23137'
# grab only entities in coal mine network
query = """

MATCH (ep:Energy_Project)-[r:SHARE*1..]->(e:Entity)
WHERE ep.energy_project_type = "Coal Mine" AND ep.status IN ["Operating", "Operating pre-retirement"]
RETURN DISTINCT id(ep) AS neo4j_id,
    ep.name AS name,
    ep.id AS id,
    CASE WHEN ep:Energy_Project THEN 1
        ELSE 0 END AS is_energy_project,
    ep.country AS country,
    CASE WHEN ep:Energy_Project THEN ep.energy_project_type
        WHEN ep:Entity THEN 'Entity'
        ELSE 'Unknown' END AS energy_project_type,
    CASE WHEN ep:Energy_Project THEN ep.status
        WHEN ep:Entity THEN 'Entity'
        ELSE 'Unknown' END AS status,
    COALESCE(ep.coal_mine_capacity, 0) AS capacity,
    COALESCE(ep.centrality_coal_mine, 0) AS centrality,
    COALESCE(ep.betweenness_coal_mine, 0) AS betweenness,
    COALESCE(ep.page_rank_coal_mine, 0) AS pagerank,
    apoc.node.degree(ep, '<SHARE') AS incoming_share_count

UNION

MATCH (ep:Energy_Project)-[r:SHARE*1..]->(e:Entity)
WHERE ep.energy_project_type = "Coal Mine" AND ep.status IN ["Operating", "Operating pre-retirement"]
UNWIND r AS rel
RETURN DISTINCT id(endNode(rel)) AS neo4j_id,
    endNode(rel).name AS name,
    endNode(rel).id AS id,
    CASE WHEN endNode(rel):Energy_Project THEN 1
        ELSE 0 END AS is_energy_project,
    endNode(rel).country AS country,
    CASE WHEN endNode(rel):Energy_Project THEN endNode(rel).energy_project_type
        WHEN endNode(rel):Entity THEN 'Entity'
        ELSE 'Unknown' END AS energy_project_type,
    CASE WHEN endNode(rel):Energy_Project THEN endNode(rel).status
        WHEN endNode(rel):Entity THEN 'Entity'
        ELSE 'Unknown' END AS status,
    COALESCE(endNode(rel).coal_mine_capacity, 0) AS capacity,
    COALESCE(endNode(rel).centrality_coal_mine, 0) AS centrality,
    COALESCE(endNode(rel).betweenness_coal_mine, 0) AS betweenness,
    COALESCE(endNode(rel).page_rank_coal_mine, 0) AS pagerank,
    apoc.node.degree(endNode(rel), '<SHARE') AS incoming_share_count

"""
df = neo4j_to_pandas(query, query_type='nodes')

  Query Results:
-------------------------
(2670, 12)


,neo4j_id,name,id,is_energy_project,country,energy_project_type,status,capacity,centrality,betweenness,pagerank,incoming_share_count
0,40853,Airly Coal Mine M0002 Operating nan,EP21706,1,Australia,Coal Mine,Operating,0,0.0,0.0,0.95,0
1,40854,"Appin Coal Mine West Cliff, Tower Colliery, Ap...",EP21707,1,Australia,Coal Mine,Operating,0,0.0,0.0,0.95,0
2,40855,Aquila Coal Mine M0006 Operating nan,EP21708,1,Australia,Coal Mine,Operating,0,0.0,0.0,0.95,0
3,40856,Ashton Coal Mine Watag M0007 Operating nan,EP21709,1,Australia,Coal Mine,Operating,0,0.0,0.0,0.95,0
4,40857,Baralaba Coal Mine Baralaba North M0008 Operat...,EP21710,1,Australia,Coal Mine,Operating,0,0.0,0.0,0.95,0


In [211]:
df.status = df.status.apply(lambda x: x.lower() if pd.notnull(x) else x)
df.capacity = df.capacity.astype(float)
df.index.name = 'index'
df.to_json('/user/projects/project-3-energy-ownership/code/Data/data_viz/test/coal_mine_nodes_8_6_24.json', orient='records')

In [6]:
# grab all relationships
query = """

MATCH (ep:Energy_Project)-[r:SHARE*]->(end:Entity)
WHERE ep.energy_project_type = "Coal Mine" AND ep.status IN ["Operating", "Operating pre-retirement"]
UNWIND r AS rel
RETURN DISTINCT id(rel) AS rel_id,
    startNode(rel).id AS source,
    endNode(rel).id AS target,
    CASE WHEN startNode(rel):Energy_Project THEN 1
        ELSE 0 END AS is_source_energy_project,
    CASE WHEN startNode(rel):Energy_Project THEN startNode(rel).energy_project_type
         WHEN startNode(rel):Entity THEN 'Entity'
         ELSE 'Unknown' END AS energy_project_type,
    CASE WHEN startNode(rel):Entity THEN 'Entity'
        WHEN startNode(rel):Energy_Project THEN startNode(rel).status
        ELSE 'Unknown' END AS status,
    rel.share AS share;

"""
df = neo4j_to_pandas(query)

  Query Results:
-------------------------
(2539, 7)


,rel_id,source,target,is_source_energy_project,energy_project_type,status,share
0,1096,EP21706,E100001001131,1,Coal Mine,Operating,100.0
1,14189,E100001001131,E100001001132,0,Entity,Entity,100.0
2,14191,E100001001132,E100001016373,0,Entity,Entity,100.0
3,14192,E100001016373,E100001001134,0,Entity,Entity,100.0
4,14188,E100001001134,E100001001135,0,Entity,Entity,100.0


In [213]:
df.status = df.status.apply(lambda x: x.lower() if pd.notnull(x) else x)
df.index.name = 'index'
df.to_json('/user/projects/project-3-energy-ownership/code/Data/data_viz/test/coal_mine_relationships_8_6_24.json', orient='records')

- steel plant

In [22]:
# all entities
query = """

MATCH (ep:Energy_Project)
WHERE ep.energy_project_type = "Steel Plant" AND ep.status IN ["operating", "operating pre-retirement"]
RETURN DISTINCT id(ep) AS neo4j_id,
    ep.name AS name,
    ep.id AS id,
    CASE WHEN ep:Energy_Project THEN 1
        ELSE 0 END AS is_energy_project,
    ep.country AS country,
    CASE WHEN ep:Energy_Project THEN ep.energy_project_type
        WHEN ep:Entity THEN 'Entity'
        ELSE 'Unknown' END AS energy_project_type,
    CASE WHEN ep:Energy_Project THEN ep.status
        WHEN ep:Entity THEN 'Entity'
        ELSE 'Unknown' END AS status,
    COALESCE(ep.steel_plant_capacity, 0) AS capacity,
    COALESCE(ep.centrality_steel_plant_, 0) AS centrality,
    COALESCE(ep.betweenness_steel_plant_, 0) AS betweenness,
    COALESCE(ep.page_rank_steel_plant_, 0) AS pagerank,
    apoc.node.degree(ep, '<SHARE') AS incoming_share_count

UNION

MATCH (ep:Energy_Project)-[r:SHARE*]->(e:Entity)
WHERE ep.energy_project_type = "Steel Plant" AND ep.status IN ["operating", "operating pre-retirement"]
UNWIND r AS rel
RETURN DISTINCT id(endNode(rel)) AS neo4j_id,
    endNode(rel).name AS name,
    endNode(rel).id AS id,
    CASE WHEN endNode(rel):Energy_Project THEN 1
        ELSE 0 END AS is_energy_project,
    endNode(rel).country AS country,
    CASE WHEN endNode(rel):Energy_Project THEN endNode(rel).energy_project_type
        WHEN endNode(rel):Entity THEN 'Entity'
        ELSE 'Unknown' END AS energy_project_type,
    CASE WHEN endNode(rel):Energy_Project THEN endNode(rel).status
        WHEN endNode(rel):Entity THEN 'Entity'
        ELSE 'Unknown' END AS status,
    COALESCE(endNode(rel).steel_plant_capacity, 0) AS capacity,
    COALESCE(endNode(rel).centrality_steel_plant, 0) AS centrality,
    COALESCE(endNode(rel).betweenness_steel_plant, 0) AS betweenness,
    COALESCE(endNode(rel).page_rank_steel_plant, 0) AS pagerank,
    apoc.node.degree(endNode(rel), '<SHARE') AS incoming_share_count

"""
df = neo4j_to_pandas(query, query_type='nodes')

  Query Results:
-------------------------
(2872, 12)


,neo4j_id,name,id,is_energy_project,country,energy_project_type,status,capacity,centrality,betweenness,pagerank,incoming_share_count
0,43173,Aba Iron and Steel Payas plant,EP24026,1,Türkiye,Steel Plant,operating,0,0.0,0.0,0.0,0
1,43175,Abinsk Electric Steel Works,EP24028,1,Russia,Steel Plant,operating,0,0.0,0.0,0.0,0
2,43176,Abul Khair Steel Sitakunda plant,EP24029,1,Bangladesh,Steel Plant,operating,0,0.0,0.0,0.0,0
3,43177,Acciaierie d'Italia Taranto steel plant,EP24030,1,Italy,Steel Plant,operating,0,0.0,0.0,0.0,0
4,43179,Acciaierie Venete Borgo Valsugana steel plant,EP24032,1,Italy,Steel Plant,operating,0,0.0,0.0,0.0,0


In [24]:
df.status = df.status.apply(lambda x: x.lower() if pd.notnull(x) else x)
df.capacity = df.capacity.astype(float)
df.index.name = 'index'
df.to_json('/user/projects/project-3-energy-ownership/code/Data/data_viz/test/steel_plant_nodes_8_6_24.json', orient='records')

In [25]:
# grab all relationships
query = """

MATCH (ep:Energy_Project)-[r:SHARE*]->(end:Entity)
WHERE ep.energy_project_type = "Steel Plant" AND ep.status IN ["operating", "operating pre-retirement"]
UNWIND r AS rel
RETURN DISTINCT id(rel) AS rel_id,
    startNode(rel).id AS source,
    endNode(rel).id AS target,
    CASE WHEN startNode(rel):Energy_Project THEN 1
        ELSE 0 END AS is_source_energy_project,
    CASE WHEN startNode(rel):Energy_Project THEN startNode(rel).energy_project_type
         WHEN startNode(rel):Entity THEN 'Entity'
         ELSE 'Unknown' END AS energy_project_type,
    CASE WHEN startNode(rel):Entity THEN 'Entity'
        WHEN startNode(rel):Energy_Project THEN startNode(rel).status
        ELSE 'Unknown' END AS status,
    rel.share AS share;

"""
df = neo4j_to_pandas(query)

  Query Results:
-------------------------
(2616, 7)


,rel_id,source,target,is_source_energy_project,energy_project_type,status,share
0,2632,EP24028,E100000130999,1,Steel Plant,operating,100.0
1,15278,E100000130999,E100000131027,0,Entity,Entity,100.0
2,2633,EP24029,E100000131068,1,Steel Plant,operating,100.0
3,15279,E100000131068,E100000131048,0,Entity,Entity,0.0
4,2634,EP24030,E100001010116,1,Steel Plant,operating,100.0


In [27]:
df.status = df.status.apply(lambda x: x.lower() if pd.notnull(x) else x)
df.index.name = 'index'
df.to_json('/user/projects/project-3-energy-ownership/code/Data/data_viz/test/steel_plant_relationships_8_6_24.json', orient='records')